# Basket QRBS with Noise

En este Notebook hago pruebas del QRBS del baloncesto (basquet_qrbs.py) para preparar las simulaciones ruidosas

In [ ]:
import sys
sys.path.append("../")
import itertools as it
import pandas as pd
import numpy as np
from qat.core import Job, Batch

In [ ]:
from selectable_qpu import SelectableQPU
qpu_selected = SelectableQPU()

In [ ]:
# myQLM qpus
from qpu.select_qpu import select_qpu
from qpu.benchmark_utils import combination_for_list
# List with the strings that should be provided for an ideal QPU
ideal_qpus = ["c", "python", "linalg", "mps", "qlmass_linalg", "qlmass_mps"]
qpu_config_c = {
    "qpu_type": ideal_qpus[2], 
}
ideal_qpu = select_qpu(qpu_config_c)

In [ ]:
from basket import basquet_qrbs

Los jugadores que voy a usar

In [ ]:
Name = ["Elias", "Blas", "Luis", "Juan", "Raul", "Cholo"]
Throws = [16, 17, 17, 15, 18, 18]
Heights = [198, 193, 188, 203, 176, 186]

## 0. Compruebo CRY

En el módulo **neasqc_qrbs/knowledge_rep.py** se implementó manualmente la puerta controlada $R_y$. Esta implementación nos da problemas al aplicar la reescritura de circuitos y los modelos de ruidos. He realizado un cambio utilizando la implementación nativa. Este código permite comprobar que ambas implementaciones son correctas (Hay que cambiar la línea correspondiente en **neasqc_qrbs/knowledge_rep.py** y re ejecutar para poder comparar resultados) 

In [ ]:
model = "bayes"
player_evaluation_ideal = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=ideal_qpu, shots=0, model=model) for t, h in zip(Throws, Heights)
]
pdf_ideal = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_ideal)]
pdf_ideal = pd.DataFrame(
    pdf_ideal,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf_ideal.sort_values(["Final_Score"], ascending=False, inplace=True)


In [ ]:
pdf_ideal.to_html().replace("\n", "")

### CRY

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Name</th>      <th>Throws</th>      <th>Height</th>      <th>Final_Score</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Elias</td>      <td>16</td>      <td>198</td>      <td>64.425626</td>    </tr>    <tr>      <th>1</th>      <td>Blas</td>      <td>17</td>      <td>193</td>      <td>59.527263</td>    </tr>    <tr>      <th>3</th>      <td>Juan</td>      <td>15</td>      <td>203</td>      <td>54.896615</td>    </tr>    <tr>      <th>5</th>      <td>Cholo</td>      <td>18</td>      <td>186</td>      <td>17.617664</td>    </tr>    <tr>      <th>2</th>      <td>Luis</td>      <td>17</td>      <td>188</td>      <td>16.860870</td>    </tr>    <tr>      <th>4</th>      <td>Raul</td>      <td>18</td>      <td>176</td>      <td>0.000000</td>    </tr>  </tbody></table>

### CRY using qlm

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Name</th>      <th>Throws</th>      <th>Height</th>      <th>Final_Score</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Elias</td>      <td>16</td>      <td>198</td>      <td>64.425626</td>    </tr>    <tr>      <th>1</th>      <td>Blas</td>      <td>17</td>      <td>193</td>      <td>59.527263</td>    </tr>    <tr>      <th>3</th>      <td>Juan</td>      <td>15</td>      <td>203</td>      <td>54.896615</td>    </tr>    <tr>      <th>5</th>      <td>Cholo</td>      <td>18</td>      <td>186</td>      <td>17.617664</td>    </tr>    <tr>      <th>2</th>      <td>Luis</td>      <td>17</td>      <td>188</td>      <td>16.860870</td>    </tr>    <tr>      <th>4</th>      <td>Raul</td>      <td>18</td>      <td>176</td>      <td>0.000000</td>    </tr>  </tbody></table>

## 1. Solución Ideal

Crea la solución con la que se debe comparar. Este código ejecuta el circuito tal cual sale del código sin optimizaciones ni re escrituras

In [ ]:
final_model_ideal = []
for model in ["cf", "fuzzy", "bayes"]:

    player_evaluation_ideal = [
        basquet_qrbs(t, h, qpu_selected, type_qpu=ideal_qpu, shots=0, model=model) for t, h in zip(Throws, Heights)
    ]
    pdf_ideal = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_ideal)]
    pdf_ideal = pd.DataFrame(
        pdf_ideal,
        columns = ["Name", "Throws", "Height", "Final_Score"]
    )
    pdf_ideal.sort_values(["Final_Score"], ascending=False, inplace=True)
    final_model_ideal.append(pdf_ideal)

Se comprueba que los 3 modelos  (**CF**, **FUZZY** y **BAYES**) den lo mismo

In [ ]:
np.isclose(final_model_ideal[0]["Final_Score"], final_model_ideal[1]["Final_Score"] )

In [ ]:
np.isclose(final_model_ideal[0]["Final_Score"], final_model_ideal[2]["Final_Score"] )

In [ ]:
final_model_ideal[0]

## 2.Circuit Rewriting

Con el código en esta sección queremos comprobar que las rescrituras del circuito dan exactamente lo mismo que el circuito original). 

La simulación es ideal pero los circuitos se rescribe utilizando:
* KAKCOMPRESION
* EXPANSION_COLLECTION

In [ ]:
rewriter_qpu = [{
    "qpu_type": ["ideal"],
    "kak_compiler" : ["ZXZ"],
    "qpu_name" : ["ideal"],
    "t_gate_1qb" : [None],
    "t_gate_2qbs" : [None],
    "t_readout": [None],
    "sim_method" : [
        {
        "sim_method": "deterministic",
        "n_samples" : None,
        "bond_dimension": None
        },
        {
        "sim_method": "stochastic",
        "n_samples" : 100, 
        "bond_dimension": None            
        },
        {
        "sim_method": "mpo",
        "n_samples" : None,
        "bond_dimension": 16
        },        
    ],
    "n_samples" : [1000],
    "depol_channel" : [{
        "active": False,
        "error_gate_1qb" : None,
        "error_gate_2qbs" : None
    }],
    "idle" : [{
        "amplitude_damping": False,
        "dephasing_channel": False,
        "t1" : None,
        "t2" : None
    }],
    "meas": [{
        "active":False,
        "readout_error": None
    }]
}]
ideal_rewriter = combination_for_list(rewriter_qpu)
ideal_rewriter_qpu = select_qpu(ideal_rewriter[0])

In [ ]:
final_model_rewriter = []
for model in ["cf", "fuzzy", "bayes"]:
    player_evaluation_ideal_rewriter = [
        basquet_qrbs(t, h, qpu_selected, type_qpu=ideal_rewriter_qpu, shots=0, model=model) for t, h in zip(Throws, Heights)
    ]
    pdf_ideal_rewriter = [[n, t, h, p["final_score"]] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_ideal_rewriter)]
    pdf_ideal_rewriter = pd.DataFrame(
        pdf_ideal_rewriter,
        columns = ["Name", "Throws", "Height", "Final_Score"]
    )
    pdf_ideal_rewriter.sort_values(["Final_Score"], ascending=False, inplace=True)
    
    final_model_rewriter.append(pdf_ideal_rewriter)

Compara las soluciones

In [ ]:
for i in range(3):
    print(np.isclose(final_model_ideal[i]["Final_Score"], final_model_rewriter[i]["Final_Score"] ))

### 2.1 Re escritura de circuitos

Explicamos las re-escrituras que hace el sistema.

Primero extraemos un circuito original cualquiera:

In [ ]:
qrbs_id = 0
island_id = 2

#Select a builder
model = "bayes"
builder = qpu_selected.BUILDERS[model]

# Get the QRBS
basket = player_evaluation_ideal[qrbs_id]["qrbs"]
# Get all the Islands of the QRBS
islas = basket._engine._islands
# Select an island
isla = islas[island_id]
#Generate circuit
circuit = isla.build(builder)
print("Initial circuit")
%qatdisplay circuit

#### KAKCOMPRESION

Esta funcionalidad agrega puertas de 1 qubit y las descompone utilizando un patrón seleccionado. Los patrones de descomposición se seleccionan de la función *list_decompositions* y pueden ser:
* 'ions' or 'rx+': $R_Z−R_X(π/2)−R_Z−R_X(π/2)−R_Z$
* 'ZXZ': $R_Z−R_X−R_Z$
* 'XZX': $R_X−R_Z−R_X$
* 'ZYZ': $R_Z−R_Y−R_Z$
* 'u3' or 'ibm': $U3$ gate


In [ ]:
from qat.plugins import KAKCompression
from qat.pbo.kak import list_decompositions

In [ ]:
list_decompositions()

A continuación comprobamos el circuito resultante cuando se aplica la funcionalidad *KAKCompression*

In [ ]:
batch_rew = Batch(jobs=[circuit.to_job()])
kak_decomposition = list_decompositions()[0]
print(kak_decomposition)
kak_plugin = KAKCompression(decomposition=kak_decomposition)
kak_plugin = kak_plugin | ideal_qpu
compilation = kak_plugin.compile(batch_rew)
compiled_circuit = compilation.jobs[0].circuit
print("Compiled circuit: KAK")
%qatdisplay compiled_circuit

#### EXPANSION_COLLECTION

Sobre el circuito resultante vamos a aplicar una segunda reescritura consistente en sustitución de patrones (vamos a hacer expansion de puertas) usando los patrones de *EXPANSION_COLLECTION* y la funcionalidad *PatternManager*. 

La que más nos interesa realmente es la expansión de las puertas Toffoli ya que los modelos de ruidos solo los definimos para puertas de 1 y 2 qubits.

In [ ]:
from qat.synthopline.compiler import EXPANSION_COLLECTION
from qat.pbo.collections import INVOLUTIONS
from qat.pbo import PatternManager

Circuito resultante despues de aplicar la coleccion de patrones de *EXPANSION_COLLECTION*

In [ ]:
batch_rew = Batch(jobs=[compiled_circuit.to_job()])
expansion_plugin = PatternManager(collections=[EXPANSION_COLLECTION[0:1]])
expansion_plugin = expansion_plugin | ideal_qpu
compilation2 = expansion_plugin.compile(batch_rew)
compiled_circuit2 = compilation2.jobs[0].circuit
print("Compiled circuit: EXPANSION")
%qatdisplay compiled_circuit2

In [ ]:
kak_decomposition = list_decompositions()[4]
kak_plugin = KAKCompression(decomposition=kak_decomposition)
kak_plugin = kak_plugin | ideal_qpu
batch_rew = Batch(jobs=[compiled_circuit2.to_job()])
compilation3 = kak_plugin.compile(batch_rew)
compiled_circuit3 = compilation3.jobs[0].circuit
%qatdisplay compiled_circuit3

### Todo junto

Comprobamos que podemos aplicar los dos plugins de reescritura utilizados (KAKCompression y PatternManager con EXPANSION_COLLECTION)

In [ ]:
%qatdisplay circuit 
batch_rew = Batch(jobs=[circuit.to_job()])
rewrite_circuit = ideal_rewriter_qpu.compile(batch_rew).jobs[0].circuit
%qatdisplay rewrite_circuit 

#### Testeo la solución

Vamos a ejecutar los circuitos generados y comprobar que lo que sale es lo mismo:

In [ ]:
EXPANSION_COLLECTION

In [ ]:
def proccess_qresults(result):
    """
    Post Process a QLM results for creating a pandas DataFrame

    Parameters
    ----------

    result : QLM results from a QLM qpu.
        returned object from a qpu submit
    qubits : int
        number of qubits
    complete : bool
        for return the complete basis state.
    """
    list_for_results = []
    for sample in result:
        list_for_results.append([
            sample.state, sample.state.lsb_int, sample.probability,
            sample.amplitude, sample.state.int,
        ])

    pdf = pd.DataFrame(
        list_for_results,
        columns=['States', "Int_lsb", "Probability", "Amplitude", "Int"]
    )
    pdf.sort_values(["Int_lsb"], inplace=True)
    return pdf    

In [ ]:
%qatdisplay circuit 

job_0 = circuit.to_circ().to_job(nbshots=0)
result_job0 = ideal_qpu.submit(job_0)
result_job0 = proccess_qresults(result_job0)

In [ ]:
%qatdisplay rewrite_circuit 

job_1 = rewrite_circuit.to_job(nbshots=0)
result_job1 = ideal_qpu.submit(job_1)
result_job1 = proccess_qresults(result_job1)

In [ ]:
np.isclose(result_job0["Probability"], result_job1["Probability"])

## 3. Noisy Simulations

Finalmente podemos aplicar el modelo de ruido sobre los diferentes circuitos generados. Para ello se utiliza la función *select_qpu* del módulo **qpu.select_qpu**. Esta función recibe un diccionario de configuración y genera la qpu ruidosa correspondiente que ya incluye los plugins de reescritura anteriormente descritos. 

In [ ]:
import json

Vamos a cargar las configuraciones de un *JSON* previamente generado:

In [ ]:
with open("qpu/qpu_noisy_deterministic.json") as json_file:
    noisy_cfg = json.load(json_file)
qpu_list = combination_for_list(noisy_cfg)

In [ ]:
len(qpu_list)

El *JSON* tiene varias posibles qpus ruidosas. Debemos seleccionar una de las posibles configuraciones:

In [ ]:
noisy_id = -1
noisy_qpu_cfg = qpu_list[noisy_id]
print(noisy_qpu_cfg)

Una vez seleccionado un diccionario de configuración se lo proporcionamos a la función *select_qpu* que nos devolverá una qpu convenientemente configurada:

In [ ]:
noisy_qpu = select_qpu(noisy_qpu_cfg)

In [ ]:
print(noisy_qpu)
# Error de las puertas de 1 qubit
print(noisy_qpu.hardware_model.gate_noise["H"].keywords["rb_eps"]  == noisy_qpu_cfg["depol_channel"]["error_gate_1qb"])
# Error de las puertas de 2 qubits
print(noisy_qpu.hardware_model.gate_noise["CNOT"].keywords["rb_eps"]  == noisy_qpu_cfg["depol_channel"]["error_gate_2qbs"])
# T1 para Amplitude Damping
print(noisy_qpu.hardware_model.idle_noise[0].T_1 == noisy_qpu_cfg["idle"]["t1"])
# Tvarphi para Dephaising
t_var_phi = 1.0 / (1.0/noisy_qpu_cfg["idle"]["t2"] - 1.0/(2* noisy_qpu_cfg["idle"]["t1"] ))
print(noisy_qpu.hardware_model.idle_noise[1].T_phi == t_var_phi)

### ATENCIÓN!!

Si usamos simulación determinista como método de simulación (*deterministic* en **sim_method** key) la simulación fallará para los modelos **CF** y **FUZZY**. En estos casos solo se puede usar simulación estocástica (*stochastic* en **sim_method** key). 

In [ ]:
model = "bayes"
shots = 0
player_evaluation_noisy = [
    basquet_qrbs(t, h, qpu_selected, type_qpu=noisy_qpu, shots=shots, model=model) for t, h in zip(Throws, Heights)
]
pdf_noisy = [[n, t, h, p] for n, t, h, p in zip(Name, Throws, Heights, player_evaluation_noisy)]
pdf_noisy = pd.DataFrame(
    pdf_noisy,
    columns = ["Name", "Throws", "Height", "Final_Score"]
)
pdf_noisy.sort_values(["Final_Score"], ascending=False, inplace=True)
pdf_noisy.reset_index(drop=True, inplace=True)
qpu_pdf = to_pdf(noisy_qpu_cfg)
qpu_pdf["shots"] = shots
qpu_pdf["model"] = model
qpu_pdf = pd.concat([qpu_pdf] * len(Throws))
qpu_pdf.reset_index(drop=True, inplace=True)
pdf = pd.concat([qpu_pdf, pdf_noisy], axis = 1)


In [ ]:
pdf